<a href="https://colab.research.google.com/github/chengsun1/NLP-orgs-markets/blob/master/bert_base_uncased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import transformers
import torch

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.3 MB/s eta 0:00:00


In [12]:
dftr = pd.read_csv('train.csv')

In [13]:
train_df, eval_df = train_test_split(dftr, test_size=0.15, stratify=dftr['label'], random_state=42)

In [14]:
# Load model directly
from transformers import AutoTokenizer, BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, padding = 'max_length', max_length = 512)
    tokens['labels'] = [int(label) for label in batch['label']]
    return tokens

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=16)
eval_dataset = eval_dataset.map(tokenize, batched=True, batch_size=16)

Map:   0%|          | 0/17898 [00:00<?, ? examples/s]

Map:   0%|          | 0/3159 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
from transformers import Trainer
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset
)

In [15]:
trainer.train()

Step,Training Loss
10,0.797300
20,0.802700
30,0.742600
40,0.886200
50,0.862400
60,0.833700
70,0.747600
80,0.794900
90,0.746300
100,0.790100


TrainOutput(global_step=11190, training_loss=0.25341953941174783, metrics={'train_runtime': 4170.2684, 'train_samples_per_second': 42.918, 'train_steps_per_second': 2.683, 'total_flos': 4.710466894900838e+16, 'train_loss': 0.25341953941174783, 'epoch': 10.0})

In [16]:
from sklearn.metrics import f1_score
import numpy as np
predictions, labels, _ = trainer.predict(eval_dataset)
preds = np.argmax(predictions, axis=1)

In [17]:
f1 = f1_score(labels, preds, average='weighted')
print(f'F1 Score: {f1}')

F1 Score: 0.6334675635116406


In [18]:
pred = pd.read_csv('test.csv')

In [20]:
pred_dataset = Dataset.from_pandas(pred)
pred_dataset = pred_dataset.map(tokenize, batched=True, batch_size=16)

Map:   0%|          | 0/8943 [00:00<?, ? examples/s]

In [21]:
pred_encoded = tokenizer(pred_dataset['text'], truncation=True, padding=True)

In [22]:
pred_dataset = Dataset.from_dict(pred_encoded)
predictions, _, _ = trainer.predict(pred_dataset)
preds = np.argmax(predictions, axis =1)

In [23]:
output = pd.read_csv('submission.csv')
output['pred_label']=preds
output.to_csv('submission_bert_base.csv', index = False)